# Data Cleaning and Processing

This Notebook will process the data and extract all the relevant information needed for analysis. This will include:
- Feature Selection
- Imputing missing values
- Removing possible Duplicates (in case it was not properly preformed in the data collection process)
- Normalizing and Encoding categorical variables

Then the cleaned data will be exported into a format for the next step of EDA.

Imputing missing values will be done using the following methods:
- Mode for categorical variables
- For `selftext` the blank values will be filled with "title_only"


## Import Data

In [63]:
## Imports
import pandas as pd

## Sklearn 
from sklearn.preprocessing import  StandardScaler
from sklearn.compose import ColumnTransformer

## NLTK
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

#Ignore warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [64]:
## Load data
agg_df = pd.read_csv('../2_EDA/data/00_master.csv')

print(agg_df.shape)
agg_df.head()

(8018, 118)


,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,...,preview,is_gallery,media_metadata,gallery_data,author_cakeday,link_flair_template_id,poll_data,crosspost_parent_list,crosspost_parent,collections
0,NaN,Army,This was a convo I had with one of my buddies ...,t2_3vmh30ad,False,NaN,0,False,If you could create a new MRE based on a Fast ...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Army,BLUF: how do you overcome imposter syndrome?\n...,t2_9mqncmmb,False,NaN,0,False,how do you even Army?,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Army,"Long story short, my estranged (soon to be ex)...",t2_ag69n7u7,False,NaN,0,False,Command Directed No-Contact Order?,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Army,"\nMy husband is 35T, and just graduated AIT. W...",t2_lb56g2zm,False,NaN,0,False,Anyone 35T?,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Army,I could use some advice on going recruiting. I...,t2_i4rellgt,False,NaN,0,False,Thinking of going recruiter as brand new E5,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Statistics

In [65]:
agg_df.columns

Index(['approved_at_utc', 'subreddit', 'selftext', 'author_fullname', 'saved',
       'mod_reason_title', 'gilded', 'clicked', 'title', 'link_flair_richtext',
       ...
       'preview', 'is_gallery', 'media_metadata', 'gallery_data',
       'author_cakeday', 'link_flair_template_id', 'poll_data',
       'crosspost_parent_list', 'crosspost_parent', 'collections'],
      dtype='object', length=118)

In [66]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8018 entries, 0 to 8017
Columns: 118 entries, approved_at_utc to collections
dtypes: bool(28), float64(25), int64(8), object(57)
memory usage: 5.7+ MB


In [67]:
agg_df.dtypes

approved_at_utc           float64
subreddit                  object
selftext                   object
author_fullname            object
saved                        bool
                           ...   
link_flair_template_id     object
poll_data                  object
crosspost_parent_list      object
crosspost_parent           object
collections                object
Length: 118, dtype: object

## Feature Selection

### Dropped Columns

In [68]:
# Drop columns: 'thumbnail_width', 'media_embed' and 74 other columns
agg_df = agg_df.drop(columns=['thumbnail_width', 'media_embed', 'author_flair_template_id', 'is_original_content', 'user_reports', 'secure_media', 'is_reddit_media_domain', 'is_meta', 'category', 'secure_media_embed', 'link_flair_text', 'can_mod_post', 'approved_by', 'is_created_from_ads_ui', 'author_premium', 'gildings', 'content_categories', 'mod_note', 'wls', 'removed_by_category', 'banned_by', 'link_flair_type', 'allow_live_comments', 'selftext_html', 'likes', 'suggested_sort', 'banned_at_utc', 'view_count', 'archived', 'is_crosspostable', 'pinned', 'all_awardings', 'awarders', 'media_only', 'can_gild', 'spoiler', 'treatment_tags', 'visited', 'removed_by', 'num_reports', 'distinguished', 'subreddit_id', 'author_is_blocked', 'mod_reason_by', 'removal_reason', 'link_flair_background_color', 'id', 'is_robot_indexable', 'report_reasons', 'discussion_type', 'send_replies', 'whitelist_status', 'contest_mode', 'mod_reports', 'author_patreon_flair', 'author_flair_text_color', 'permalink', 'parent_whitelist_status', 'stickied', 'url', 'subreddit_subscribers', 'created_utc', 'num_crossposts', 'media', 'post_hint', 'url_overridden_by_dest', 'preview', 'is_gallery', 'media_metadata', 'gallery_data', 'author_cakeday', 'link_flair_template_id', 'poll_data', 'crosspost_parent_list', 'crosspost_parent', 'collections', 'approved_at_utc', 'saved', 'mod_reason_title', 'gilded', 'clicked', 'hidden', 'pwls', 'link_flair_css_class', 'downs', 'thumbnail_height', 'top_awarded_type', 'hide_score', 'name', 'quarantine', 'link_flair_text_color', 'author_flair_background_color', 'subreddit_type', 'total_awards_received'])

In [69]:
agg_df.isnull().sum().sort_values(ascending=False)


author_flair_css_class     7629
author_flair_text          6019
thumbnail                  2004
selftext                   1135
author_flair_richtext        32
author_fullname              32
author_flair_type            32
created                       0
num_comments                  0
author                        0
locked                        0
over_18                       0
no_follow                     0
domain                        0
subreddit                     0
is_self                       0
edited                        0
score                         0
ups                           0
upvote_ratio                  0
subreddit_name_prefixed       0
link_flair_richtext           0
title                         0
is_video                      0
dtype: int64

### Author Flair CSS Class

In [70]:
#Replace null values with 'blank' in 'author_flair_css_class'
agg_df['author_flair_css_class'].value_counts(ascending=True)


author_flair_css_class
recruiter                  1
electronicwarfare          1
spc                        1
none                       1
camper                     2
jag                        2
Chaplain                   2
chemical                   2
darkgoldenrod              2
medicalservice             3
AquisitionCorps            3
civilaffairs               4
a                          5
cavalry                    5
psycologicaloperations     6
Cyber                      6
publicaffairs              6
airdefenseartillery        8
medicalspecialist          8
armor                      9
militarypolice             9
transportation            10
fieldartillery            12
quartermaster             12
adjutantgeneral           13
engineer                  16
medical                   18
ordnance                  23
aviation                  28
militaryintelligence      33
signal                    37
infantry                  50
blank                     51
Name: count, dtype: 

### Author flair text

In [71]:
agg_df['author_flair_text'].value_counts()
agg_df.drop(columns=['author_flair_text'], inplace=True)

### Thumbnail

In [72]:
agg_df['thumbnail'].value_counts()
agg_df.drop(columns=['thumbnail'], inplace=True)

### Self Text

In [73]:
agg_df['selftext'].value_counts()

selftext
So when fresh faced idiot babies (2nd Lts) go through the Basic School there is a peer evaluation process that encourages back stabbing and other such nonsense. Part of it is a list of attributes and one-word descriptors you have to assign to everyone in your platoon. I actively campaigned to get the “springbutt” moniker. I was proud of 85% of the platoon ranking that as the first thing they thought of when looking at me. \n\nAnyway, I’m trying to find that form, anyone have a copy?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

### Author Flair richtext

In [74]:
agg_df['author_flair_richtext'].value_counts()
agg_df.drop(columns=['author_flair_richtext'], inplace=True)

### Author Full Name

In [75]:
agg_df['author_fullname'].value_counts()
agg_df.drop(columns=['author_fullname'], inplace=True)

### Author Flair Type

In [76]:
agg_df['author_flair_type'].value_counts()


author_flair_type
text        7203
richtext     783
Name: count, dtype: int64

### Original Columns

In [77]:
print(agg_df.shape)
agg_df.dtypes

(8018, 20)


subreddit                   object
selftext                    object
title                       object
link_flair_richtext         object
subreddit_name_prefixed     object
upvote_ratio               float64
ups                          int64
score                        int64
edited                      object
author_flair_css_class      object
is_self                       bool
created                    float64
author_flair_type           object
domain                      object
no_follow                     bool
over_18                       bool
locked                        bool
author                      object
num_comments                 int64
is_video                      bool
dtype: object

## Imputing missing Values

### Author Flair CSS Class

In [78]:
agg_df['author_flair_css_class'].fillna('blank', inplace=True)
agg_df['author_flair_css_class'].value_counts()
print(f'Author Flair CSS Class Null is: {agg_df["author_flair_css_class"].isnull().sum()}')

Author Flair CSS Class Null is: 0


### Author Flair Type

In [79]:
agg_df['author_flair_type'].fillna('unknown', inplace=True)
agg_df['author_flair_type'].value_counts()
print(f'Author Flair Type Null is: {agg_df["author_flair_type"].isnull().sum()}')

Author Flair Type Null is: 0


### Self Text

In [80]:
agg_df['selftext'].fillna('title_only', inplace=True)
agg_df['selftext'].value_counts()
print(f'Self Text Null is: {agg_df["selftext"].isnull().sum()}')

Self Text Null is: 0


### Values Count

In [81]:
agg_df.isnull().sum().sort_values(ascending=False)

subreddit                  0
selftext                   0
num_comments               0
author                     0
locked                     0
over_18                    0
no_follow                  0
domain                     0
author_flair_type          0
created                    0
is_self                    0
author_flair_css_class     0
edited                     0
score                      0
ups                        0
upvote_ratio               0
subreddit_name_prefixed    0
link_flair_richtext        0
title                      0
is_video                   0
dtype: int64

## Removing Duplicates

In [82]:
agg_df.duplicated(subset=['author']).sum()

5302

In [83]:
agg_df.duplicated(subset=['title']).sum()

4693

In [84]:
agg_df.duplicated(subset=['selftext']).sum()

5055

In [85]:
agg_df.drop_duplicates(subset=['selftext'], inplace=True)

agg_df.duplicated(subset=['selftext']).sum()

0

In [86]:
agg_df.shape

(2963, 20)

In [87]:
agg_df.isnull().sum().sort_values(ascending=False)

subreddit                  0
selftext                   0
num_comments               0
author                     0
locked                     0
over_18                    0
no_follow                  0
domain                     0
author_flair_type          0
created                    0
is_self                    0
author_flair_css_class     0
edited                     0
score                      0
ups                        0
upvote_ratio               0
subreddit_name_prefixed    0
link_flair_richtext        0
title                      0
is_video                   0
dtype: int64

## Normalizing and Encoding

In [88]:
agg_df.dtypes

subreddit                   object
selftext                    object
title                       object
link_flair_richtext         object
subreddit_name_prefixed     object
upvote_ratio               float64
ups                          int64
score                        int64
edited                      object
author_flair_css_class      object
is_self                       bool
created                    float64
author_flair_type           object
domain                      object
no_follow                     bool
over_18                       bool
locked                        bool
author                      object
num_comments                 int64
is_video                      bool
dtype: object

In [89]:
agg_df.head(15)

,subreddit,selftext,title,link_flair_richtext,subreddit_name_prefixed,upvote_ratio,ups,score,edited,author_flair_css_class,is_self,created,author_flair_type,domain,no_follow,over_18,locked,author,num_comments,is_video
0,Army,This was a convo I had with one of my buddies ...,If you could create a new MRE based on a Fast ...,[],r/army,1.00,3,3,False,blank,True,1.703376e+09,text,self.army,False,False,False,Mariostar16,3,False
1,Army,BLUF: how do you overcome imposter syndrome?\n...,how do you even Army?,[],r/army,0.57,1,1,False,airdefenseartillery,True,1.703375e+09,richtext,self.army,True,False,False,Ok-Philosopher5533,4,False
2,Army,"Long story short, my estranged (soon to be ex)...",Command Directed No-Contact Order?,[],r/army,1.00,2,2,False,blank,True,1.703373e+09,text,self.army,False,False,False,skywri8293,4,False
3,Army,"\nMy husband is 35T, and just graduated AIT. W...",Anyone 35T?,[],r/army,0.84,4,4,False,blank,True,1.703372e+09,text,self.army,False,False,False,Deep-Ad1124,8,False
4,Army,I could use some advice on going recruiting. I...,Thinking of going recruiter as brand new E5,[],r/army,1.00,2,2,False,blank,True,1.703371e+09,text,self.army,False,False,False,Classcelicos,6,False
5,Army,At my new duty station in Massachusetts and al...,"Well everyone, I’m alone for the holidays and ...",[],r/army,0.89,29,29,False,medical,True,1.703368e+09,richtext,self.army,False,False,False,BeanieWeenie-Burrito,24,False
6,Army,Does anyone have experience with submitting th...,DD Form 368 has started its way up. Looking fo...,[],r/army,0.75,2,2,False,blank,True,1.703368e+09,text,self.army,False,False,False,Aggravating_Tart_299,7,False
7,Army,Also what does ‘Standard Excess’ refer to for ...,What does ‘Permanent Change of Station’ refer ...,[],r/army,0.83,8,8,False,blank,True,1.703366e+09,text,self.army,False,False,False,Bambonoy,5,False
8,Army,Any tips or advice. I'm in osut about to turn ...,Advice for 17 year old,[],r/army,0.76,8,8,1703369322.0,blank,True,1.703366e+09,text,self.army,False,False,False,churnedslinger,30,False
9,Army,Would anyone be able to describe the process o...,Space Force NCO to Army Warrant Officer,[],r/army,0.60,1,1,False,blank,True,1.703365e+09,text,self.army,True,False,False,cantthinkofaname1010,4,False


In [90]:
def preprocess_dataframe(df):
    # Normalize numerical columns
    numerical_cols = ['upvote_ratio', 'ups', 'score', 'num_comments']
    df[numerical_cols] = StandardScaler().fit_transform(df[numerical_cols])

    # Convert boolean columns to binary
    boolean_cols = ['edited', 'is_self', 'no_follow', 'over_18', 'locked', 'is_video']
    for col in boolean_cols:
        df[col] = df[col].apply(lambda x: 1 if x == True or x == 'True' else 0)

    # One-hot encode categorical columns
    categorical_cols = ['author_flair_css_class', 'author_flair_type', 'domain']
    df = pd.get_dummies(df, columns=categorical_cols, dtype= int)

    # Process 'created' column (timestamp)
    df['created'] = pd.to_datetime(df['created'], unit='s')
    df['hour_of_day'] = df['created'].dt.hour
    df['day_of_week'] = df['created'].dt.dayofweek
    df = df.drop('created', axis=1)

    return df

# Cleaned Data

In [91]:
processed_df = preprocess_dataframe(agg_df)

print(processed_df.shape)
processed_df.head(10)

(2963, 74)


,subreddit,selftext,title,link_flair_richtext,subreddit_name_prefixed,upvote_ratio,ups,score,edited,is_self,...,domain_self.USMC,domain_self.army,domain_self.bipolar,domain_self.schizophrenia,domain_stripes.com,domain_v.redd.it,domain_youtu.be,domain_youtube.com,hour_of_day,day_of_week
0,Army,This was a convo I had with one of my buddies ...,If you could create a new MRE based on a Fast ...,[],r/army,0.663463,-0.261143,-0.261143,0,1,...,0,1,0,0,0,0,0,0,23,5
1,Army,BLUF: how do you overcome imposter syndrome?\n...,how do you even Army?,[],r/army,-1.599996,-0.288827,-0.288827,0,1,...,0,1,0,0,0,0,0,0,23,5
2,Army,"Long story short, my estranged (soon to be ex)...",Command Directed No-Contact Order?,[],r/army,0.663463,-0.274985,-0.274985,0,1,...,0,1,0,0,0,0,0,0,23,5
3,Army,"\nMy husband is 35T, and just graduated AIT. W...",Anyone 35T?,[],r/army,-0.178754,-0.247301,-0.247301,0,1,...,0,1,0,0,0,0,0,0,22,5
4,Army,I could use some advice on going recruiting. I...,Thinking of going recruiter as brand new E5,[],r/army,0.663463,-0.274985,-0.274985,0,1,...,0,1,0,0,0,0,0,0,22,5
5,Army,At my new duty station in Massachusetts and al...,"Well everyone, I’m alone for the holidays and ...",[],r/army,0.084438,0.098748,0.098748,0,1,...,0,1,0,0,0,0,0,0,21,5
6,Army,Does anyone have experience with submitting th...,DD Form 368 has started its way up. Looking fo...,[],r/army,-0.652502,-0.274985,-0.274985,0,1,...,0,1,0,0,0,0,0,0,21,5
7,Army,Also what does ‘Standard Excess’ refer to for ...,What does ‘Permanent Change of Station’ refer ...,[],r/army,-0.231393,-0.191933,-0.191933,0,1,...,0,1,0,0,0,0,0,0,21,5
8,Army,Any tips or advice. I'm in osut about to turn ...,Advice for 17 year old,[],r/army,-0.599863,-0.191933,-0.191933,0,1,...,0,1,0,0,0,0,0,0,21,5
9,Army,Would anyone be able to describe the process o...,Space Force NCO to Army Warrant Officer,[],r/army,-1.442080,-0.288827,-0.288827,0,1,...,0,1,0,0,0,0,0,0,20,5


### Output processed df in ANALYSIS folder for further analysis

In [92]:
#output to csv
processed_df.to_csv('../3_ANALYSIS/data/processed_data.csv', index=False)

In [93]:
processed_df.shape

(2963, 74)